In [ ]:
import astropy
import sep
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
fname = 'hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits'
hdu_list = fits.open(fname)
hdu_list.info()
print(hdu_list)

In [ ]:
image_data = astropy.io.fits.getdata(fname)    #obtains the actual data from the fits file

#show image
m, s = np.mean(image_data), np.std(image_data) 
plt.imshow(image_data, interpolation = 'nearest', cmap = 'gray', vmin = m-s, vmax = m+s,  origin = 'lower')
plt.colorbar()

In [ ]:
image_data = hdu_list[0].data
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation = 'nearest', cmap='gray', vmin = m-s, vmax = m+s, origin='lower')
plt.colorbar()

In [ ]:
image_data = image_data.byteswap().newbyteorder()
bkg = sep.Background(image_data, bw=64, bh=64, fw=3, fh=3)
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back() #evaluates background as a 2d array, like the original image
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')

In [ ]:
#background noise as a 2d array
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')

In [ ]:
#subtract the background
image_data_sub = image_data-bkg

In [ ]:
thresh = 50
objects = sep.extract(image_data_sub, thresh, err=bkg.globalrms*1.5)
#what should thresh be, depends on image?
print('Number of objects detected: ', len(objects))

In [ ]:
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(image_data_sub), np.std(image_data_sub)
im = ax.imshow(image_data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=10*objects['a'][i], height=10*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

### Kehan Zou: Convert to AB magnitudes

In [ ]:
flux, fluxerr, flag = sep.sum_circle(image_data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
for i in range(len(objects)):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
AB_mag = [0] * len(objects)
for nmbr in range(len(objects)):
    
    AB_mag[nmbr]= -2.5*np.math.log10(flux[nmbr]/4)
    print(AB_mag[nmbr])

In [ ]:
histogram = plt.hist(AB_mag, bins=len(AB_mag))
plt.xlim(-6.,6.)